# Ridge regression main files
Code editor: Xinyi Li, Yinchuan Li. Date: 2019.2.20.

The code is run on Google's Colaboratory with Python 3.

Paper: Mid-LSTM meets Mid-ARMA: deep learning for midterm stock prediction.


## Import packages

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("drive/download data/sp500new")
# !ls
from google.colab import files
import os
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm 
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131322 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [0]:
def stock_ridge_loop (filename):

  split = (0.85);
  sequence_length=60;
  normalise= True
  batch_size=60;
  input_dim=4
  input_timesteps=59
  neurons=60
  epochs=2
  prediction_len=60
  dense_output=4
  window_size=60

  dataframe = pd.read_csv(filename)
  
  #pre stock put on first
  cols = ['Close_y', 'Volume_y','Close_x'];#corr will be add on later


  len_dataframe=dataframe.shape[0]
  corr_num=int(len_dataframe/sequence_length)
  remainder=len_dataframe-corr_num*sequence_length

  # caculate corr table
  corr_win=[]
  corr=np.zeros((len_dataframe))
  for i in range(0,corr_num):
    stock1=[]
    stock2=[]
    for j in range(i*sequence_length,i*sequence_length+sequence_length):
      stock1.append(dataframe[cols[0]][j])
      stock2.append(dataframe[cols[2]][j])
    corr_win.append(np.corrcoef(stock1, stock2)[0,1])
    for j in range(i*sequence_length,i*sequence_length+sequence_length):
      corr[j]=corr_win[i]


  corr_win_remainder=[] 
  stock1_remainder=[]
  stock2_remainder=[] 
  for k in range(0,remainder):
    stock1_remainder.append(dataframe[cols[0]][corr_num*sequence_length+k])
    stock2_remainder.append(dataframe[cols[2]][corr_num*sequence_length+k])
  corr_win_remainder.append(np.corrcoef(stock1_remainder, stock2_remainder)[0,1])
  for q in range(0,remainder):
    corr[corr_num*sequence_length+q]=corr_win_remainder[0]

  i_split = int(len(dataframe) * split)
  data_train = dataframe.get(cols).values[:i_split]
  data_test  = dataframe.get(cols).values[i_split:]
  len_train  = len(data_train)
  len_test   = len(data_test)
  len_train_windows = None

  corr_df=pd.DataFrame(corr)
  data_corr_train=corr_df.values[:i_split]
  data_corr_test=corr_df.values[i_split:]

  #get_test_data  ##############################################################
  data_windows = []
  for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
  data_windows = np.array(data_windows).astype(float)

  # get original y_test
  y_test_ori = data_windows[:, -1, [0]]

  window_data=data_windows
  win_num=window_data.shape[0]
  row_num=window_data.shape[1]
  col_num=window_data.shape[2]
  normalised_data = []
  record_min=[]
  record_max=[]

  for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
  normalised_data=np.array(normalised_data)

  corr_windows = []
  for i in range(len_test - sequence_length):
    corr_windows.append(data_corr_test[i:i+sequence_length])
  corr_windows = np.array(corr_windows).astype(float)

  get_test_data=[]
  for win_i in range(0,win_num):
    df1=pd.DataFrame(normalised_data[win_i,:,:])
    df1['corr']=corr_windows[win_i,:,:]
    df2=df1.values
    get_test_data.append(df2)
  get_test_data=np.array(get_test_data)  
 
  data_windows=get_test_data
  x_test = data_windows[:, :-1]
  y_test = data_windows[:, -1, [0]]

  #get_train_data #############################################################
  data_windows = []
  for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
  data_windows = np.array(data_windows).astype(float)

  window_data=data_windows
  win_num=window_data.shape[0]
  row_num=window_data.shape[1]
  col_num=window_data.shape[2]

  normalised_data = []
  for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
  normalised_data=np.array(normalised_data)

  corr_windows_train = []
  for i in range(len_train - sequence_length):
    corr_windows_train.append(data_corr_train[i:i+sequence_length])
  corr_windows_train = np.array(corr_windows_train).astype(float)

  get_train_data=[]
  for win_i in range(0,win_num):
    df1=pd.DataFrame(normalised_data[win_i,:,:])
    df1['corr']=corr_windows_train[win_i,:,:]
    df2=df1.values
    get_train_data.append(df2)
  get_train_data=np.array(get_train_data)  

  data_windows=get_train_data
  x_train = data_windows[:, :-1]
  y_train = data_windows[:, -1]

  ##3 ridge regression
  x_train_ri=pd.DataFrame(x_train[:,:,0])
  y_train_ri=pd.DataFrame(y_train[:,0])
  x_test_ri=pd.DataFrame(x_test[:,:,0])
  y_test_ri=pd.DataFrame(y_test[:,0])
  y_test_ori_ri=pd.DataFrame(y_test_ori[:,0])

  clf = Ridge(alpha=1.0)
  clf.fit(x_train_ri, y_train_ri) 

  data=x_test[:,:,0]
  
  prediction_seqs = []
  pre_win_num=int(len(data)/prediction_len)
  window_size=sequence_length

  for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []

    for j in range(0,prediction_len):
      predicted.append(clf.predict(curr_frame[newaxis,:])[0])
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

  #de_predicted
  de_predicted=[]
  len_pre_win=int(len(data)/prediction_len)
  len_pre=prediction_len
  
  m=0
  for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j]*record_max[m]+record_min[m])
      m=m+1

  error = []
  diff=y_test.shape[0]-prediction_len*pre_win_num

  for i in range(y_test_ori.shape[0]-diff):
      error.append(y_test_ori[i,] - de_predicted[i])

  squaredError = []
  absError = []
  for val in error:
      squaredError.append(val * val) 
      absError.append(abs(val))

  error_percent=[]
  for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

  mean_error_percent=sum(error_percent) / len(error_percent)
  accuracy=1-mean_error_percent

  MSE=sum(squaredError) / len(squaredError)
  return MSE,accuracy,y_test_ori,de_predicted

In [0]:
filename=np.load('filename_delete_sort.npy')

result_ridge_df=pd.DataFrame(columns=('index','stock','MSE','accuracy','true','predict'))
n=len(filename)
                           
for i in range(0,n):
  index=i
  stock=filename[i]
  result=stock_ridge_loop(filename[i])
  MSE=result[0]
  accuracy=result[1]
  true=result[2]
  predict=result[3]
  result_ridge_df=result_ridge_df.append(pd.DataFrame({'index':[index],
                                                     'stock':[stock],
                                                     'MSE':[MSE],
                                                     'accuracy':[accuracy],
                                                     'true':[true],
                                                     'predict':[predict]}),ignore_index=True)
  print(i)
  np.save('ridge_d0.npy',result_ridge_df)
  result_ridge_df.to_csv('ridge_d0.csv')

In [0]:
#Reshape data
L4_all=np.load('ridge_d0.npy')
L4_all=pd.DataFrame(L4_all)
L4_all.columns=['MSE','accuracy','index','predict','stock','TRUE']

#accuracy only
filename=np.load('filename_delete_sort.npy')
n=len(filename)
len_pre=360

result_df=pd.DataFrame(columns=('index','stock','TRUE','predict','accuracy','MSE'))
for i in range(0,n):
  index=i
  stock=filename[i]
  #TRUE
  t=[]
  for j in range(0,len_pre):
    t.append(L4_all['TRUE'][i][j][0])
  TRUE=t
  #predict
  t=[]
  for j in range(0,len_pre):
    t.append(L4_all['predict'][i][j][0])
  predict=t
  #accuracy
  accuracy=[]
  for j in range(0,len_pre):
    t=abs(TRUE[j]-predict[j])/TRUE[j]
    t1=1-t
    accuracy.append(t1)
  accuracy=accuracy
  #MSE
  MSE=L4_all['MSE'][i][0]
  result_df=result_df.append(pd.DataFrame({'index':index,'stock':[stock],
                                           'TRUE':[TRUE],
                                           'predict':[predict],
                                          'accuracy':[accuracy],
                                          'MSE':[MSE]}),
                             ignore_index=True)
  print(i)
np.save('ridge_r_new.npy',result_df)

In [0]:
##Mean MPA of all stocks
ridge_r=np.load('ridge_r_new.npy')
ridge_r=pd.DataFrame(ridge_r)
ridge_r.columns=['MSE','TRUE','accuracy','index','predict','stock']

n=451
avg_accuracy1=[]
for i in range(0,360):
  t1=0
  for j in range(0,n):
    t1=t1+ridge_r['accuracy'][j][i]
  t1=t1/n
  avg_accuracy1.append(t1)

half1=[]
for i in range(0,6): 
  half1.extend(avg_accuracy1[60*(i+1)-30:60*(i+1)]) 
mean1=pd.DataFrame(half1).mean()[0]

print('Ridge regression Mean MPA:',mean1)

Ridge regression Mean MPA: 0.9252519275147825


## Portfolio allocation

###Mean variance portfolio allocation based on ridge regression (0-60 days) , asset 1.

In [0]:
LH_dph=np.load('ridge_r_new.npy')
LH_dph=pd.DataFrame(LH_dph)
LH_dph.columns = ['MSE','TRUE','accuracy','index','predict','stock']

test_win=6
pre_len1=60
pre_len2=60
stock_len=451
rf=0.015#risk free rate

df_all=pd.DataFrame(columns=('index','return_pre','variance_pre','sharp_pre',
                            'return_true','variance_true','sharp_true',))

filename=np.load('filename_delete_sort.npy')
df = pd.DataFrame()

####cumulative
for k in range(0,test_win):
  n=stock_len
  
  for i in range(0,n):
    t=LH_dph['predict'][i][k*pre_len1:k*pre_len1+pre_len1]
    t1=[]
    for j in range(0,pre_len1):
      t1.append(t[j])
    t=t1
    df[filename[i]]=t  
  data1=df
  log_returns = np.log(data1 / data1.shift(1))
  ret_index = (1 + log_returns).cumprod()
  
  choose_name=[]
  choose_index=[]
  for i in range(0,n):
    if ret_index[filename[i]][59]>1.15:
      choose_name.append(filename[i])
      choose_index.append(i)
      
  #choose data
  m=len(choose_index)
  data2=data1.iloc[:,choose_index]
  log_returns = np.log(data2 / data2.shift(1))
  
  rets = log_returns
  year_ret = rets.mean() * 252
  year_volatility = rets.cov() * 252
  number_of_assets = m
  weights = np.random.random(number_of_assets)
  weights /= np.sum(weights)
  
  def statistics(weights):        
    weights = np.array(weights)
    pret = np.sum(rets.mean() * weights) * 252
    pvol = np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))
    return np.array([pret, pvol, (pret-rf) / pvol])
  
  def min_func_sharpe(weights):
    return -statistics(weights)[2]
    
  bnds = tuple((0,1) for x in range(number_of_assets))
  cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

  opts = sco.minimize(min_func_sharpe, 
                      number_of_assets * [1. / number_of_assets,],
                      method='SLSQP', 
                      bounds=bnds, 
                      constraints=cons)
  
  weights_pre=opts['x']
  
  ##check return
  df2 = pd.DataFrame()
  for i in choose_index:
    t=LH_dph['TRUE'][i][k*pre_len1:k*pre_len1+pre_len1]
    t1=[]
    for j in range(0,pre_len1):
      t1.append(t[j])
    t= t1
    df2['ture'+filename[i]]=t
  data3=df2 
  log_returns_true = np.log(data3 / data3.shift(1))
  
  rets_true = log_returns_true
  year_ret_true = rets_true.mean() * 252
  year_volatility_true = rets_true.cov() * 252
  number_of_assets = m  #real asset number
  
  def statistics_true(weights):        
    weights = np.array(weights)
    pret = np.sum(rets_true.mean() * weights) * 252
    pvol = np.sqrt(np.dot(weights.T, np.dot(rets_true.cov() * 252, weights)))
    return np.array([pret, pvol, (pret-rf) / pvol])
  
  index=k
  return_pre=statistics(opts['x'])[0]
  variance_pre=statistics(opts['x'])[1]
  sharp_pre=statistics(opts['x'])[2]
  return_true=statistics_true(opts['x'])[0]
  variance_true=statistics_true(opts['x'])[1]
  sharp_true=statistics_true(opts['x'])[2]
  
  df_all=df_all.append(pd.DataFrame({'index':[index],
                                     'return_pre':[return_pre],
                                     'variance_pre':[variance_pre],
                                     'sharp_pre':[sharp_pre],
                                     'return_true':[ return_true],
                                     'variance_true':[variance_true],
                                    'sharp_true':[sharp_true],}),ignore_index=True)
#   print('stock number:',n)
#   print('count',k)
#   print('choose number',m)
#   print('initial random weight',weights)
#   print('pre weights',opts['x'])
print('Mean variance portfolio allocation based on ridge regression (0-60 days) , all stocks: \n',df_all)
#   print('weight',opts['x'].round(3))


Mean variance portfolio allocation based on ridge regression (0-60 days) , all stocks: 
   index  return_pre  return_true  sharp_pre  sharp_true  variance_pre  \
0     0    0.796437     0.783196  16.881991    4.670287      0.046288   
1     1    0.918241     1.222854  26.208353   10.367646      0.034464   
2     2    0.744868     0.568767   8.884582    2.144261      0.082150   
3     3    0.798062     0.692123  43.344174    4.628260      0.018066   
4     4    0.761791     0.807338  43.119256    7.820320      0.017319   
5     5    0.785273     0.755630  12.762811    3.337458      0.060353   

   variance_true  
0       0.164486  
1       0.116502  
2       0.258255  
3       0.146302  
4       0.101318  
5       0.221914  


###Minimum variance portfolio allocation based on ridge regression (0-60 days) , asset 1.

In [0]:
LH_dph=np.load('ridge_r_new.npy')
LH_dph=pd.DataFrame(LH_dph)
LH_dph.columns = ['MSE','TRUE','accuracy','index','predict','stock']

test_win=6
pre_len1=60
pre_len2=60
stock_len=451
rf=0.015#risk free rate

df_all=pd.DataFrame(columns=('index','return_pre','variance_pre','sharp_pre',
                            'return_true','variance_true','sharp_true',))

filename=np.load('filename_delete_sort.npy')
df = pd.DataFrame()

####cumulative
for k in range(0,test_win):
  n=stock_len
  
  for i in range(0,n):
    t=LH_dph['predict'][i][k*pre_len1:k*pre_len1+pre_len1]
    t1=[]
    for j in range(0,pre_len1):
      t1.append(t[j])
    t=t1
    df[filename[i]]=t  
  data1=df
  log_returns = np.log(data1 / data1.shift(1))
  ret_index = (1 + log_returns).cumprod()
  
  choose_name=[]
  choose_index=[]
  for i in range(0,n):
    if ret_index[filename[i]][59]>1.15:
      choose_name.append(filename[i])
      choose_index.append(i)
      
  #choose data
  m=len(choose_index)
  data2=data1.iloc[:,choose_index]
  log_returns = np.log(data2 / data2.shift(1))
  
  rets = log_returns
  year_ret = rets.mean() * 252
  year_volatility = rets.cov() * 252
  number_of_assets = m
  weights = np.random.random(number_of_assets)
  weights /= np.sum(weights)
  
  def statistics(weights):        
    weights = np.array(weights)
    pret = np.sum(rets.mean() * weights) * 252
    pvol = np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))
    return np.array([pret, pvol, (pret-rf) / pvol])
  
  def min_func_sharpe(weights):
    return statistics(weights)[1]
    
  bnds = tuple((0,1) for x in range(number_of_assets))
  cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

  opts = sco.minimize(min_func_sharpe, 
                      number_of_assets * [1. / number_of_assets,],
                      method='SLSQP', 
                      bounds=bnds, 
                      constraints=cons)
  
  weights_pre=opts['x']
  
  ##check return
  df2 = pd.DataFrame()
  for i in choose_index:
    t=LH_dph['TRUE'][i][k*pre_len1:k*pre_len1+pre_len1]
    t1=[]
    for j in range(0,pre_len1):
      t1.append(t[j])
    t= t1
    df2['ture'+filename[i]]=t
  data3=df2 
  log_returns_true = np.log(data3 / data3.shift(1))
  
  rets_true = log_returns_true
  year_ret_true = rets_true.mean() * 252
  year_volatility_true = rets_true.cov() * 252
  number_of_assets = m  #real asset number
  
  def statistics_true(weights):        
    weights = np.array(weights)
    pret = np.sum(rets_true.mean() * weights) * 252
    pvol = np.sqrt(np.dot(weights.T, np.dot(rets_true.cov() * 252, weights)))
    return np.array([pret, pvol, (pret-rf) / pvol])
  
  index=k
  return_pre=statistics(opts['x'])[0]
  variance_pre=statistics(opts['x'])[1]
  sharp_pre=statistics(opts['x'])[2]
  return_true=statistics_true(opts['x'])[0]
  variance_true=statistics_true(opts['x'])[1]
  sharp_true=statistics_true(opts['x'])[2]
  
  df_all=df_all.append(pd.DataFrame({'index':[index],
                                     'return_pre':[return_pre],
                                     'variance_pre':[variance_pre],
                                     'sharp_pre':[sharp_pre],
                                     'return_true':[ return_true],
                                     'variance_true':[variance_true],
                                    'sharp_true':[sharp_true],}),ignore_index=True)
#   print('stock number:',n)
#   print('count',k)
#   print('choose number',m)
#   print('initial random weight',weights)
#   print('pre weights',opts['x'])
print('Minimum variance portfolio allocation based on ridge regression (0-60 days) , all stocks: \n',df_all)
#   print('weight',opts['x'].round(3))


Minimum variance portfolio allocation based on ridge regression (0-60 days) , all stocks: 
   index  return_pre  return_true  sharp_pre  sharp_true  variance_pre  \
0     0    0.753332     0.733688  16.425097    4.322717      0.044951   
1     1    0.872483     1.158078  25.569300    9.588978      0.033536   
2     2    0.708565     0.605436   8.662348    2.343849      0.080067   
3     3    0.716640     0.590743  41.110297    4.185277      0.017067   
4     4    0.718246     0.768684  41.825964    7.685020      0.016814   
5     5    0.756526     0.676397  12.522136    3.100964      0.059217   

   variance_true  
0       0.166258  
1       0.119208  
2       0.251909  
3       0.137564  
4       0.098072  
5       0.213288  


###Mean variance portfolio allocation based on ridge regression (0-60 days) , asset 2.

In [0]:
filename=np.load('filename_delete_sort.npy')
sort=np.load('new_sortname.npy')

LH_dph=np.load('ridge_r_new.npy')
LH_dph=pd.DataFrame(LH_dph)
LH_dph.columns = ['MSE','TRUE','accuracy','index','predict','stock']
LH_dph_ori=LH_dph

top=50

LH_dph=[]
for i in range(0,451):
  for j in range(0,top):
    if LH_dph_ori['stock'][i]==sort[j]:
      LH_dph.append(LH_dph_ori.iloc[i,])
      
LH_dph=np.array(LH_dph)
LH_dph=pd.DataFrame(LH_dph)
LH_dph.columns = ['MSE','TRUE','accuracy','index','predict','stock']

test_win=6
pre_len1=60
pre_len2=60
rf=0.015
stock_len=top

df_all=pd.DataFrame(columns=('index','return_pre','variance_pre','sharp_pre',
                            'return_true','variance_true','sharp_true',))

filename=np.load('filename_delete_sort.npy')
df = pd.DataFrame()

rets_true_all=pd.DataFrame()
####cumulative
for k in range(0,test_win):
  n=stock_len
  
  for i in range(0,n):
    t=LH_dph['predict'][i][k*pre_len1:k*pre_len1+pre_len1]
    t1=[]
    for j in range(0,pre_len1):
      t1.append(t[j])
    t=t1
    df[filename[i]]=t  
  data1=df
  log_returns = np.log(data1 / data1.shift(1))
  ret_index = (1 + log_returns).cumprod()
  
  choose_name=[]
  choose_index=[]
  for i in range(0,n):
    if ret_index[filename[i]][59]>1.03:
        choose_name.append(filename[i])
        choose_index.append(i)

  #choose data
  m=len(choose_index)
  data2=data1.iloc[:,choose_index]
  log_returns = np.log(data2 / data2.shift(1))
  
  rets = log_returns
  number_of_assets = m
  weights = np.random.random(number_of_assets)
  weights /= np.sum(weights)
  
  def statistics(weights):        
    weights = np.array(weights)
    pret = np.sum(rets.mean() * weights) * 252
    pvol = np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))
    return np.array([pret, pvol, (pret-rf) / pvol])
  
  def min_func_sharpe(weights):
    return -statistics(weights)[2]
  
  bnds = tuple((0,1) for x in range(number_of_assets))
  cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

  opts = sco.minimize(min_func_sharpe, 
                      number_of_assets * [1. / number_of_assets,],
                      method='SLSQP', 
                      bounds=bnds, 
                      constraints=cons)
  
  weights_pre=opts['x']
  
  ##check return
  df2 = pd.DataFrame()
  for i in choose_index:
    t=LH_dph['TRUE'][i][k*pre_len1:k*pre_len1+pre_len1]
    t1=[]
    for j in range(0,pre_len1):
      t1.append(t[j])
    t= t1
    df2['ture'+filename[i]]=t
  data3=df2 
  log_returns_true = np.log(data3 / data3.shift(1))
  
  rets_true = log_returns_true
  a=rets_true_all.append(rets_true,ignore_index=True) 
  number_of_assets = m  #real asset number
  
  def statistics_true(weights):        
    weights = np.array(weights)
    pret = np.sum(rets_true.mean() * weights) * 252
    pvol = np.sqrt(np.dot(weights.T, np.dot(rets_true.cov() * 252, weights)))
    return np.array([pret, pvol, (pret-rf) / pvol])
  
  index=k
  return_pre=statistics(opts['x'])[0]
  variance_pre=statistics(opts['x'])[1]
  sharp_pre=statistics(opts['x'])[2]
  return_true=statistics_true(opts['x'])[0]
  variance_true=statistics_true(opts['x'])[1]
  sharp_true=statistics_true(opts['x'])[2]
  
  df_all=df_all.append(pd.DataFrame({'index':[index],
                                     'return_pre':[return_pre],
                                     'variance_pre':[variance_pre],
                                     'sharp_pre':[sharp_pre],
                                     'return_true':[ return_true],
                                     'variance_true':[variance_true],
                                    'sharp_true':[sharp_true],}),ignore_index=True)
#   print('stock number:',n)
#   print('count',k)
#   print('choose number',m)
#   print('initial random weight',weights)
#   print('pre weights',opts['x'])
#   print('weight',opts['x'].round(3))
print('Mean variance portfolio allocation based on ridge regression (0-60 days) , asset 2: \n',df_all)


Mean variance portfolio allocation based on ridge regression (0-60 days) , asset 2: 
   index  return_pre  return_true  sharp_pre  sharp_true  variance_pre  \
0     0    0.220958     0.251311  18.170321    2.685436      0.011335   
1     1    0.228817     0.210891  12.966416    2.819985      0.016490   
2     2    0.355898     0.247504  10.967833    1.291678      0.031082   
3     3    0.437225     0.256502  25.027221    1.995265      0.016871   
4     4    0.375218     0.331941  34.846071    3.424521      0.010337   
5     5    0.267181     0.142459  15.124748    0.874061      0.016673   

   variance_true  
0       0.087997  
1       0.069465  
2       0.180001  
3       0.121037  
4       0.092551  
5       0.145824  
